<a href="https://www.bigdatauniversity.com"><img src="https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width="400" align="center"></a>

<h1 align="center"><font size="5">COLLABORATIVE FILTERING</font></h1>

Recommendation systems are a collection of algorithms used to recommend items to users based on information taken from the user. These systems have become ubiquitous can be commonly seen in online stores, movies databases and job finders. In this notebook, we will explore recommendation systems based on Collaborative Filtering and implement simple version of one using Python and the Pandas library.

<h1>Table of contents</h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#ref1">Acquiring the Data</a></li>
        <li><a href="#ref2">Preprocessing</a></li>
        <li><a href="#ref3">Collaborative Filtering</a></li>
    </ol>
</div>
<br>
<hr>



<a id="ref1"></a>
# Acquiring the Data

To acquire and extract the data, simply run the following Bash scripts:  
Dataset acquired from [GroupLens](http://grouplens.org/datasets/movielens/). Lets download the dataset. To download the data, we will use **`!wget`** to download it from IBM Object Storage.  
__Did you know?__ When it comes to Machine Learning, you will likely be working with large datasets. As a business, where can you host your data? IBM is offering a unique opportunity for businesses, with 10 Tb of IBM Cloud Object Storage: [Sign up now for free](http://cocl.us/ML0101EN-IBM-Offer-CC)

In [ ]:
!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
print('unziping ...')
#!unzip -o -j moviedataset.zip 

Now you're ready to start working with the data!

<hr>

<a id="ref2"></a>
# Preprocessing

First, let's get all of the imports out of the way:

In [1]:
#Dataframe manipulation library
import pandas as pd
#Math functions, we'll only need the sqrt function so let's import only that
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Now let's read each file into their Dataframes:

In [2]:
#Storing the movie information into a pandas dataframe
movies_df = pd.read_csv('moviedataset/ml-latest/movies.csv')
#Storing the user information into a pandas dataframe
ratings_df = pd.read_csv('moviedataset/ml-latest/ratings.csv')

Let's also take a peek at how each of them are organized:

In [3]:
#Head is a function that gets the first N rows of a dataframe. N's default is 5.
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


So each movie has a unique ID, a title with its release year along with it (Which may contain unicode characters) and several different genres in the same field. Let's remove the year from the title column and place it into its own one by using the handy [extract](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.extract.html#pandas.Series.str.extract) function that Pandas has.

Let's remove the year from the __title__ column by using pandas' replace function and store in a new __year__ column.

In [4]:
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())

Let's look at the result!

In [5]:
movies_df.head()


,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


With that, let's also drop the genres column since we won't need it for this particular recommendation system.

In [6]:
#Dropping the genres column
movies_df = movies_df.drop('genres', 1)

Here's the final movies dataframe:

In [7]:
movies_df.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


<br>

Next, let's look at the ratings dataframe.

In [8]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


Every row in the ratings dataframe has a user id associated with at least one movie, a rating and a timestamp showing when they reviewed it. We won't be needing the timestamp column, so let's drop it to save on memory.

In [9]:
#Drop removes a specified row or column from a dataframe
ratings_df = ratings_df.drop('timestamp', 1)

Here's how the final ratings Dataframe looks like:

In [10]:
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


<hr>

<a id="ref3"></a>
# Collaborative Filtering

Now, time to start our work on recommendation systems. 

The first technique we're going to take a look at is called __Collaborative Filtering__, which is also known as __User-User Filtering__. As hinted by its alternate name, this technique uses other users to recommend items to the input user. It attempts to find users that have similar preferences and opinions as the input and then recommends items that they have liked to the input. There are several methods of finding similar users (Even some making use of Machine Learning), and the one we will be using here is going to be based on the __Pearson Correlation Function__.

<img src="https://ibm.box.com/shared/static/1ql8cbwhtkmbr6nge5e706ikzm5mua5w.png" width=800px>


The process for creating a User Based recommendation system is as follows:
- Select a user with the movies the user has watched
- Based on his rating to movies, find the top X neighbours 
- Get the watched movie record of the user for each neighbour.
- Calculate a similarity score using some formula
- Recommend the items with the highest score


Let's begin by creating an input user to recommend movies to:

Notice: To add more movies, simply increase the amount of elements in the userInput. Feel free to add more in! Just be sure to write it in with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' .

In [11]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


#### Add movieId to input user
With the input complete, let's extract the input movies's ID's from the movies dataframe and add them into it.

We can achieve this by first filtering out the rows that contain the input movies' title and then merging this subset with the input dataframe. We also drop unnecessary columns for the input to save memory space.

In [18]:
inputID = movies_df[movies_df['title'].isin(inputMovies['title']).tolist()]
inputMovies = pd.merge(inputID, inputMovies)
inputMovies = inputMovies.drop('year', axis=1)
inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


In [12]:
#Filtering out the movies by title
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('year', 1)
#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.
inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


#### The users who has seen the same movies
Now with the movie ID's in our input, we can now get the subset of users that have watched and reviewed the movies in our input.


In [13]:
#Filtering out users that have watched movies that the input has watched and storing it
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset

,userId,movieId,rating
19,4,296,4.0
441,12,1968,3.0
479,13,2,2.0
531,13,1274,5.0
681,14,296,2.0
...,...,...,...
22883679,247738,296,4.0
22884132,247751,1,4.0
22884142,247751,296,4.0
22884164,247751,1274,5.0


We now group up the rows by user ID.

In [51]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = userSubset.groupby(['userId'])
userSubsetGroup

lets look at one of the users, e.g. the one with userID=1130

In [68]:
userSubsetGroup.get_group(12)

,userId,movieId,rating
441,12,1968,3.0


In [65]:
for i in userSubsetGroup:
    print(i[0]) # Column index = 0 is the userId

4
12
13
14
15
17
18
20
23
26
28
31
32
34
37
38
39
40
45
47
48
50
53
54
55
56
57
58
59
60
63
64
66
70
75
78
85
88
89
90
94
95
98
99
104
106
107
109
110
114
115
116
117
118
120
121
122
124
128
129
130
131
132
135
136
137
140
148
149
150
151
152
155
158
159
160
163
164
166
170
174
175
176
178
179
180
183
184
187
192
198
199
201
202
205
206
208
210
212
216
217
219
222
223
224
226
227
228
229
233
234
235
237
240
241
243
244
251
255
256
257
258
259
261
264
265
269
272
274
275
276
277
282
285
287
288
293
294
296
302
303
304
305
309
310
312
313
317
319
320
321
322
323
326
328
329
330
332
334
337
340
341
345
347
348
349
350
351
352
355
358
360
361
362
363
365
366
368
373
375
382
383
385
386
388
390
391
392
393
394
396
397
400
401
403
404
405
406
407
408
409
414
416
422
424
430
432
434
436
447
448
449
450
451
454
461
463
465
467
470
472
474
476
478
479
480
481
483
484
486
488
489
491
493
494
495
497
505
506
507
508
513
519
522
523
526
528
529
530
531
533
534
535
536
537
538
540
542
544
546
547
5

3850
3854
3857
3859
3861
3862
3864
3865
3869
3870
3872
3875
3880
3881
3882
3883
3884
3886
3888
3890
3891
3893
3895
3896
3897
3899
3900
3903
3906
3909
3910
3911
3915
3916
3917
3918
3922
3923
3925
3926
3928
3929
3930
3931
3932
3934
3937
3943
3945
3946
3948
3952
3957
3958
3959
3960
3962
3963
3964
3966
3967
3969
3971
3976
3977
3978
3980
3981
3983
3984
3987
3988
3992
3993
3994
3995
3997
3998
3999
4001
4002
4005
4008
4009
4011
4015
4016
4017
4019
4020
4021
4023
4024
4025
4030
4033
4034
4035
4036
4037
4039
4040
4043
4046
4048
4050
4056
4057
4060
4067
4068
4070
4071
4072
4076
4079
4082
4086
4087
4089
4091
4094
4095
4099
4101
4102
4104
4105
4107
4110
4113
4115
4117
4118
4119
4121
4122
4123
4124
4125
4128
4130
4132
4137
4138
4139
4143
4144
4145
4146
4147
4148
4149
4151
4155
4157
4160
4161
4162
4163
4164
4168
4174
4177
4178
4179
4183
4186
4189
4190
4191
4193
4194
4196
4199
4201
4202
4206
4207
4208
4211
4212
4214
4217
4218
4219
4222
4223
4226
4228
4232
4234
4235
4237
4240
4241
4242
4245
4246
4248


8215
8216
8217
8218
8222
8224
8225
8226
8228
8229
8230
8232
8234
8238
8241
8242
8243
8245
8248
8251
8253
8255
8256
8258
8259
8261
8263
8264
8268
8270
8272
8276
8277
8278
8282
8284
8285
8287
8288
8289
8291
8293
8294
8295
8296
8299
8300
8303
8306
8308
8309
8311
8312
8316
8325
8327
8329
8330
8333
8335
8337
8338
8339
8342
8347
8348
8349
8350
8355
8356
8360
8361
8362
8363
8368
8370
8372
8375
8376
8377
8378
8379
8381
8382
8384
8386
8389
8394
8396
8397
8398
8399
8400
8401
8405
8406
8409
8411
8414
8415
8417
8420
8427
8428
8429
8432
8437
8438
8441
8444
8445
8447
8449
8453
8455
8457
8459
8464
8465
8468
8472
8476
8477
8479
8480
8481
8485
8486
8489
8491
8492
8494
8496
8503
8505
8506
8507
8509
8511
8514
8515
8516
8517
8519
8524
8526
8528
8529
8532
8534
8538
8541
8543
8551
8557
8558
8559
8561
8562
8565
8570
8572
8573
8574
8576
8581
8582
8583
8584
8586
8587
8592
8598
8601
8605
8610
8613
8614
8617
8618
8619
8621
8623
8625
8626
8627
8628
8630
8631
8635
8638
8642
8644
8645
8646
8648
8653
8654
8656
8659


12453
12456
12457
12458
12460
12461
12465
12469
12470
12473
12478
12479
12480
12481
12483
12485
12487
12490
12492
12493
12494
12495
12498
12499
12502
12504
12505
12507
12508
12510
12511
12512
12513
12514
12515
12516
12518
12520
12521
12522
12523
12525
12526
12528
12535
12536
12538
12540
12542
12544
12548
12554
12559
12560
12563
12565
12566
12567
12568
12571
12575
12577
12578
12582
12588
12589
12597
12600
12601
12605
12607
12611
12612
12614
12616
12617
12618
12620
12621
12623
12625
12628
12629
12630
12638
12639
12640
12641
12644
12645
12648
12649
12650
12652
12654
12657
12659
12662
12665
12667
12668
12669
12670
12671
12672
12673
12676
12679
12684
12685
12689
12690
12691
12692
12694
12697
12698
12701
12707
12708
12709
12712
12713
12715
12717
12720
12721
12722
12727
12728
12730
12731
12732
12733
12735
12736
12739
12742
12745
12746
12747
12748
12749
12750
12752
12755
12758
12761
12762
12763
12768
12769
12770
12772
12775
12779
12781
12784
12785
12786
12788
12791
12792
12793
12794
12795
1279

15460
15461
15465
15466
15467
15469
15470
15472
15473
15474
15478
15479
15480
15481
15483
15484
15487
15488
15492
15497
15498
15499
15504
15505
15506
15508
15509
15510
15511
15513
15515
15517
15520
15521
15522
15523
15524
15525
15526
15527
15528
15529
15530
15532
15533
15535
15540
15542
15544
15547
15548
15553
15558
15560
15561
15563
15564
15566
15569
15571
15574
15575
15577
15581
15585
15590
15592
15595
15597
15601
15603
15606
15610
15611
15612
15613
15615
15616
15618
15619
15621
15622
15625
15627
15629
15632
15633
15635
15637
15641
15644
15647
15649
15654
15661
15664
15667
15668
15669
15670
15673
15676
15679
15680
15682
15683
15685
15686
15687
15691
15694
15695
15698
15700
15701
15702
15704
15707
15708
15709
15715
15716
15717
15718
15720
15721
15723
15725
15726
15729
15730
15733
15737
15738
15742
15743
15750
15751
15752
15753
15757
15759
15760
15761
15765
15766
15770
15784
15785
15786
15790
15791
15796
15801
15802
15803
15805
15806
15809
15810
15811
15813
15814
15816
15818
15819
1582

...

233354
233357
233358
233359
233362
233363
233366
233370
233371
233372
233373
233374
233375
233376
233378
233379
233381
233382
233383
233384
233386
233389
233390
233392
233394
233395
233397
233400
233401
233405
233406
233407
233408
233409
233413
233415
233416
233417
233419
233420
233421
233425
233426
233430
233432
233442
233444
233445
233446
233448
233451
233454
233457
233458
233459
233460
233461
233462
233463
233465
233466
233467
233468
233469
233470
233471
233472
233473
233475
233476
233478
233480
233482
233483
233484
233487
233488
233491
233492
233493
233494
233502
233507
233508
233509
233511
233514
233517
233523
233524
233526
233530
233532
233534
233535
233536
233537
233539
233544
233545
233547
233550
233552
233553
233554
233556
233557
233558
233563
233564
233565
233566
233567
233570
233571
233572
233573
233575
233576
233577
233578
233579
233585
233587
233589
233596
233598
233599
233604
233605
233606
233607
233608
233612
233613
233615
233616
233618
233619
233621
233622
233623
233629

236204
236205
236206
236207
236209
236211
236212
236214
236215
236216
236217
236220
236225
236226
236227
236229
236230
236231
236232
236233
236235
236236
236237
236238
236239
236240
236241
236242
236244
236246
236249
236250
236251
236252
236253
236255
236257
236259
236264
236266
236267
236269
236270
236271
236274
236275
236277
236279
236280
236285
236286
236287
236288
236289
236290
236291
236294
236296
236297
236298
236299
236301
236303
236305
236306
236307
236309
236311
236317
236320
236321
236322
236323
236324
236325
236329
236332
236335
236336
236345
236348
236350
236352
236353
236356
236357
236358
236360
236362
236363
236365
236366
236370
236371
236372
236374
236375
236377
236378
236380
236381
236387
236391
236393
236394
236396
236398
236400
236405
236407
236408
236410
236412
236413
236414
236415
236416
236417
236418
236419
236420
236421
236423
236425
236426
236427
236428
236429
236432
236433
236434
236435
236436
236437
236438
236440
236441
236443
236444
236447
236452
236453
236455

238981
238982
238985
238986
238993
238996
238997
238999
239000
239003
239004
239005
239007
239011
239014
239015
239016
239018
239020
239026
239029
239030
239031
239032
239034
239037
239038
239040
239041
239044
239049
239050
239051
239053
239054
239056
239057
239061
239063
239067
239068
239070
239071
239072
239073
239077
239078
239079
239080
239081
239082
239083
239087
239088
239092
239093
239094
239098
239101
239105
239106
239110
239111
239120
239121
239122
239124
239126
239130
239134
239135
239139
239144
239145
239147
239149
239155
239157
239161
239162
239163
239164
239165
239167
239172
239177
239178
239182
239183
239185
239186
239188
239189
239192
239199
239200
239201
239203
239204
239205
239206
239209
239210
239212
239216
239217
239218
239219
239223
239225
239226
239227
239229
239232
239234
239235
239236
239240
239242
239243
239244
239245
239246
239247
239248
239250
239256
239257
239260
239262
239266
239268
239269
239272
239274
239278
239279
239283
239284
239286
239287
239288
239293

241976
241977
241978
241980
241981
241982
241983
241984
241989
241993
241994
241995
241997
241999
242000
242001
242003
242005
242009
242010
242012
242014
242015
242017
242018
242019
242020
242021
242022
242025
242028
242030
242033
242034
242040
242041
242043
242046
242048
242049
242058
242059
242060
242061
242063
242066
242068
242069
242070
242071
242072
242073
242074
242075
242077
242080
242082
242084
242090
242091
242092
242095
242099
242102
242103
242104
242105
242106
242109
242110
242111
242112
242116
242118
242120
242122
242130
242131
242132
242136
242138
242140
242142
242146
242148
242150
242153
242154
242155
242156
242157
242160
242162
242165
242167
242168
242170
242172
242177
242179
242180
242186
242188
242189
242190
242191
242193
242195
242199
242201
242203
242204
242205
242207
242209
242210
242212
242217
242218
242221
242224
242229
242231
242232
242233
242234
242237
242242
242243
242245
242247
242254
242256
242257
242259
242260
242263
242264
242268
242271
242272
242273
242276

244821
244823
244827
244828
244830
244831
244833
244836
244839
244842
244847
244848
244852
244853
244855
244856
244857
244859
244863
244866
244867
244868
244869
244870
244871
244873
244874
244875
244880
244881
244882
244884
244888
244892
244893
244898
244899
244900
244906
244907
244910
244913
244914
244915
244916
244919
244920
244923
244924
244930
244933
244935
244938
244939
244941
244943
244944
244945
244946
244947
244951
244953
244954
244956
244959
244960
244961
244962
244970
244971
244972
244973
244974
244975
244976
244977
244979
244981
244987
244989
244990
244992
244993
244999
245001
245002
245003
245005
245006
245007
245008
245010
245012
245016
245017
245019
245021
245022
245029
245031
245033
245040
245041
245043
245048
245049
245050
245051
245053
245054
245056
245057
245058
245059
245068
245071
245073
245081
245082
245083
245084
245085
245088
245091
245094
245095
245096
245101
245103
245107
245113
245114
245115
245118
245121
245122
245125
245126
245127
245129
245133
245135
245136

Let's also sort these groups so the users that share the most movies in common with the input have higher priority. This provides a richer recommendation since we won't go through every single user.

In [69]:
#Sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True) # x[1] refer to the movieId column
userSubsetGroup

[(75,
        userId  movieId  rating
  7507      75        1     5.0
  7508      75        2     3.5
  7540      75      296     5.0
  7633      75     1274     4.5
  7673      75     1968     5.0),
 (106,
        userId  movieId  rating
  9083     106        1     2.5
  9084     106        2     3.0
  9115     106      296     3.5
  9198     106     1274     3.0
  9238     106     1968     3.5),
 (686,
         userId  movieId  rating
  61336     686        1     4.0
  61337     686        2     3.0
  61377     686      296     4.0
  61478     686     1274     4.0
  61569     686     1968     5.0),
 (815,
         userId  movieId  rating
  73747     815        1     4.5
  73748     815        2     3.0
  73922     815      296     5.0
  74362     815     1274     3.0
  74678     815     1968     4.5),
 (1040,
         userId  movieId  rating
  96689    1040        1     3.0
  96690    1040        2     1.5
  96733    1040      296     3.5
  96859    1040     1274     3.0
  96922    1

Now lets look at the first user

In [23]:
userSubsetGroup[0:3]

[(75,
        userId  movieId  rating
  7507      75        1     5.0
  7508      75        2     3.5
  7540      75      296     5.0
  7633      75     1274     4.5
  7673      75     1968     5.0),
 (106,
        userId  movieId  rating
  9083     106        1     2.5
  9084     106        2     3.0
  9115     106      296     3.5
  9198     106     1274     3.0
  9238     106     1968     3.5),
 (686,
         userId  movieId  rating
  61336     686        1     4.0
  61337     686        2     3.0
  61377     686      296     4.0
  61478     686     1274     4.0
  61569     686     1968     5.0)]

#### Similarity of users to input user
Next, we are going to compare all users (not really all !!!) to our specified user and find the one that is most similar.  
we're going to find out how similar each user is to the input through the __Pearson Correlation Coefficient__. It is used to measure the strength of a linear association between two variables. The formula for finding this coefficient between sets X and Y with N values can be seen in the image below. 

Why Pearson Correlation?

Pearson correlation is invariant to scaling, i.e. multiplying all elements by a nonzero constant or adding any constant to all elements. For example, if you have two vectors X and Y,then, pearson(X, Y) == pearson(X, 2 * Y + 3). This is a pretty important property in recommendation systems because for example two users might rate two series of items totally different in terms of absolute rates, but they would be similar users (i.e. with similar ideas) with similar rates in various scales .

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation")

The values given by the formula vary from r = -1 to r = 1, where 1 forms a direct correlation between the two entities (it means a perfect positive correlation) and -1 forms a perfect negative correlation. 

In our case, a 1 means that the two users have similar tastes while a -1 means the opposite.

We will select a subset of users to iterate through. This limit is imposed because we don't want to waste too much time going through every single user.

In [70]:
userSubsetGroup = userSubsetGroup[0:100]

Now, we calculate the Pearson Correlation between input user and subset group, and store it in a dictionary, where the key is the user Id and the value is the coefficient


In [34]:
for name, group in userSubsetGroup:
    print('name',name,'group\n', group)

name 75 group
       userId  movieId  rating
7507      75        1     5.0
7508      75        2     3.5
7540      75      296     5.0
7633      75     1274     4.5
7673      75     1968     5.0
name 106 group
       userId  movieId  rating
9083     106        1     2.5
9084     106        2     3.0
9115     106      296     3.5
9198     106     1274     3.0
9238     106     1968     3.5
name 686 group
        userId  movieId  rating
61336     686        1     4.0
61337     686        2     3.0
61377     686      296     4.0
61478     686     1274     4.0
61569     686     1968     5.0
name 815 group
        userId  movieId  rating
73747     815        1     4.5
73748     815        2     3.0
73922     815      296     5.0
74362     815     1274     3.0
74678     815     1968     4.5
name 1040 group
        userId  movieId  rating
96689    1040        1     3.0
96690    1040        2     1.5
96733    1040      296     3.5
96859    1040     1274     3.0
96922    1040     1968     4.0
na

842634    9101     1968     2.5
name 9358 group
         userId  movieId  rating
863664    9358        1     4.0
863665    9358        2     2.0
863710    9358      296     5.0
863877    9358     1274     4.5
863974    9358     1968     3.0
name 9663 group
         userId  movieId  rating
886074    9663        1     4.0
886075    9663        2     3.5
886157    9663      296     5.0
886372    9663     1274     3.5
886512    9663     1968     3.0
name 9994 group
         userId  movieId  rating
923978    9994        1     5.0
923979    9994        2     3.0
924026    9994      296     5.0
924194    9994     1274     5.0
924283    9994     1968     3.5
name 10248 group
         userId  movieId  rating
948970   10248        1     4.0
948971   10248        2     3.0
949003   10248      296     2.5
949106   10248     1274     2.0
949166   10248     1968     3.5
name 10315 group
         userId  movieId  rating
955563   10315        1     4.0
955564   10315        2     3.0
955634   10315   

1652105   17666     1968     3.5
name 17735 group
          userId  movieId  rating
1660801   17735        1     3.0
1660802   17735        2     1.5
1660852   17735      296     3.0
1660987   17735     1274     2.0
1661064   17735     1968     4.0
name 17742 group
          userId  movieId  rating
1661838   17742        1     5.0
1661839   17742        2     3.0
1661888   17742      296     5.0
1661984   17742     1274     3.0
1662068   17742     1968     4.0
name 17757 group
          userId  movieId  rating
1664066   17757        1     3.5
1664067   17757        2     3.0
1664145   17757      296     5.0
1664330   17757     1274     3.5
1664456   17757     1968     3.5
name 17854 group
          userId  movieId  rating
1674302   17854        1     5.0
1674303   17854        2     4.0
1674366   17854      296     5.0
1674572   17854     1274     4.0
1674708   17854     1968     5.0
name 17897 group
          userId  movieId  rating
1678321   17897        1     5.0
1678322   17897    

In [71]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

inputMovies = inputMovies.sort_values(by='movieId')
#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    # *** inputMovies = inputMovies.sort_values(by='movieId') Don't have to do it iterately
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2) / float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0


In [72]:
pearsonCorrelationDict.items()

dict_items([(75, 0.8272781516947562), (106, 0.5860090386731182), (686, 0.8320502943378437), (815, 0.5765566601970551), (1040, 0.9434563530497265), (1130, 0.2891574659831201), (1502, 0.8770580193070299), (1599, 0.4385290096535153), (1625, 0.716114874039432), (1950, 0.179028718509858), (2065, 0.4385290096535153), (2128, 0.5860090386731196), (2432, 0.1386750490563073), (2791, 0.8770580193070299), (2839, 0.8204126541423674), (2948, -0.11720180773462392), (3025, 0.45124262819713973), (3040, 0.89514359254929), (3186, 0.6784622064861935), (3271, 0.26989594817970664), (3429, 0.0), (3734, -0.15041420939904673), (4099, 0.05860090386731196), (4208, 0.29417420270727607), (4282, -0.4385290096535115), (4292, 0.6564386345361464), (4415, -0.11183835382312353), (4586, -0.9024852563942795), (4725, -0.08006407690254357), (4818, 0.4885967564883424), (5104, 0.7674257668936507), (5165, -0.4385290096535153), (5547, 0.17200522903844556), (6082, -0.04728779924109591), (6207, 0.9615384615384616), (6366, 0.65779

In [74]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF

,0
75,0.827278
106,0.586009
686,0.832050
815,0.576557
1040,0.943456
...,...
17854,0.537086
17897,0.877058
17944,0.271385
18301,0.298381


In [75]:
# Create dataframe format
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF

,similarityIndex,userId
0,0.827278,75
1,0.586009,106
2,0.832050,686
3,0.576557,815
4,0.943456,1040
...,...,...
95,0.537086,17854
96,0.877058,17897
97,0.271385,17944
98,0.298381,18301


#### The top x similar users to input user
Now let's get the top 50 users that are most similar to the input.

In [77]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers

,similarityIndex,userId
64,0.961678,12325
34,0.961538,6207
55,0.961538,10707
67,0.960769,13053
4,0.943456,1040
59,0.937614,11769
62,0.929294,12120
80,0.903584,15157
70,0.895144,13366
17,0.895144,3040


Now, let's start recommending movies to the input user.

#### Rating of selected users to all movies
We're going to do this by taking the weighted average of the ratings of the movies using the Pearson Correlation as the weight. But to do this, we first need to get the movies watched by the users in our __pearsonDF__ from the ratings dataframe and then store their correlation in a new column called _similarityIndex". This is achieved below by merging of these two tables.

In [79]:
# Merge 2 df that is topUsers and ratings_df
# left_on='userId', right_on='userId', how='inner' ???
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating

,similarityIndex,userId,movieId,rating
0,0.961678,12325,1,3.5
1,0.961678,12325,2,1.5
2,0.961678,12325,3,3.0
3,0.961678,12325,5,0.5
4,0.961678,12325,6,2.5
...,...,...,...,...
47235,0.576557,815,146350,2.5
47236,0.576557,815,146656,3.5
47237,0.576557,815,148238,2.5
47238,0.576557,815,148626,3.5


Now all we need to do is simply multiply the movie rating by its weight (The similarity index), then sum up the new ratings and divide it by the sum of the weights.

We can easily do this by simply multiplying two columns, then grouping up the dataframe by movieId and then dividing two columns:

It shows the idea of all similar users to candidate movies for the input user:

In [81]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating

,similarityIndex,userId,movieId,rating,weightedRating
0,0.961678,12325,1,3.5,3.365874
1,0.961678,12325,2,1.5,1.442517
2,0.961678,12325,3,3.0,2.885035
3,0.961678,12325,5,0.5,0.480839
4,0.961678,12325,6,2.5,2.404196
...,...,...,...,...,...
47235,0.576557,815,146350,2.5,1.441392
47236,0.576557,815,146656,3.5,2.017948
47237,0.576557,815,148238,2.5,1.441392
47238,0.576557,815,148626,3.5,2.017948


In [82]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,38.376281,140.800834
2,38.376281,96.656745
3,10.253981,27.254477
4,0.929294,2.787882
5,11.723262,27.151751


In [83]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.668955,1
2,2.518658,2
3,2.657941,3
4,3.000000,4
5,2.316058,5


Now let's sort it and see the top 20 movies that the algorithm recommended!

In [84]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
5073,5.0,5073
3329,5.0,3329
2284,5.0,2284
26801,5.0,26801
6776,5.0,6776
6672,5.0,6672
3759,5.0,3759
3769,5.0,3769
3775,5.0,3775


In [85]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title,year
2200,2284,Bandit Queen,1994
3243,3329,"Year My Voice Broke, The",1987
3669,3759,Fun and Fancy Free,1947
3679,3769,Thunderbolt and Lightfoot,1974
3685,3775,Make Mine Music,1946
4978,5073,"Son's Room, The (Stanza del figlio, La)",2001
6563,6672,War Photographer,2001
6667,6776,Lagaan: Once Upon a Time in India,2001
9064,26801,Dragon Inn (Sun lung moon hak chan),1992
18106,90531,Shame,2011


### Advantages and Disadvantages of Collaborative Filtering

##### Advantages
* Takes other user's ratings into consideration
* Doesn't need to study or extract information from the recommended item
* Adapts to the user's interests which might change over time

##### Disadvantages
* Approximation function can be slow
* There might be a low of amount of users to approximate
* Privacy issues when trying to learn the user's preferences

<h2>Want to learn more?</h2>

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: <a href="http://cocl.us/ML0101EN-SPSSModeler">SPSS Modeler</a>

Also, you can use Watson Studio to run these notebooks faster with bigger datasets. Watson Studio is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, Watson Studio enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of Watson Studio users today with a free account at <a href="https://cocl.us/ML0101EN_DSX">Watson Studio</a>

<h3>Thanks for completing this lesson!</h3>

<h4>Author:  <a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a></h4>
<p><a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>, PhD is a Data Scientist in IBM with a track record of developing enterprise level applications that substantially increases clients’ ability to turn data into actionable knowledge. He is a researcher in data mining field and expert in developing advanced analytic methods like machine learning and statistical modelling on large datasets.</p>

<hr>

<p>Copyright &copy; 2018 <a href="https://cocl.us/DX0108EN_CC">Cognitive Class</a>. This notebook and its source code are released under the terms of the <a href="https://bigdatauniversity.com/mit-license/">MIT License</a>.</p>